#  Zillow Project: Data Preparation
---

## MVP Objectives
- Remove/Impute missing values.
- Create functions to reproduce prepared dataset
- Move functions to a seperate file named `prepare.py`


In [28]:
# Import libraries to manipulate data structures and visualize numeric data.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
# Read in the zillow dataset and assign to a variable
df = pd.read_csv('zillow.csv')

In [30]:
# My first dataset with +2 million rows :)
df.shape

(2985217, 59)

In [31]:
# Initial look at our zillow dataset
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


Our dataset has a bunch of missing values! This is only the first 5 rows! Let's see how many values are missing in each column.

In [32]:
# Using `isnull()` and `notnull()` we can calculate the number of missing values and non-null values.
nulls = df.isnull().sum()
non_nulls = df.notnull().sum()

# Add missing values and non-null values together to get the total number values in each column.
total_values = nulls + non_nulls

# Create a variable to store the percentage of missing values in each column.
pct_missing = (nulls/total_values).sort_values(ascending=False)

# Perform formatting to clearly see the percentage of missing values in each column.
pct_missing_chart = pct_missing.apply("{0:.2%}".format)

# Display table to the user showing the percentage of missing values in each column.
print('Percentage of values missing per column')
print('-' * 39)
print(f"{pct_missing_chart}")

Percentage of values missing per column
---------------------------------------
storytypeid                     99.95%
basementsqft                    99.95%
yardbuildingsqft26              99.91%
fireplaceflag                   99.83%
architecturalstyletypeid        99.80%
typeconstructiontypeid          99.77%
finishedsquarefeet13            99.74%
buildingclasstypeid             99.57%
pooltypeid10                    99.43%
decktypeid                      99.42%
finishedsquarefeet6             99.28%
poolsizesum                     99.06%
pooltypeid2                     98.89%
hashottuborspa                  98.32%
taxdelinquencyflag              98.11%
taxdelinquencyyear              98.11%
yardbuildingsqft17              97.27%
finishedsquarefeet15            93.63%
finishedsquarefeet50            93.17%
finishedfloor1squarefeet        93.17%
fireplacecnt                    89.51%
threequarterbathnbr             89.40%
pooltypeid7                     83.05%
poolcnt                

Before we move ahead let's drop columns with more than __9.14%__ of values missing. Imputing values in columns with > 9.14% of values missing is a waste of time. We need to make our dataset _Robust™_.
- If we can impute values in columns: `lotsizesquarefeet`, `finishedsquarefeet12`, great. If not, we can drop them.

In [33]:
columns_to_drop = pct_missing[pct_missing > .3358].index.to_list()

In [34]:
df_subset = df.drop(columns=columns_to_drop)

In [35]:
# My eyes are happy
df_subset.sample(10)

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
601089,601089,11637574,3.0,3.0,3.0,2153.0,2153.0,6037.0,3.0,34068827.0,...,3101.0,96086.0,0.0,1924.0,390413.0,719447.0,2016.0,329034.0,8393.33,6.037701e+13
1580744,1580744,12509266,3.0,4.0,3.0,2301.0,2301.0,6037.0,3.0,33794576.0,...,3101.0,96238.0,0.0,2012.0,183081.0,609576.0,2016.0,426495.0,7352.16,6.037575e+13
1558194,1558194,12297844,2.0,4.0,2.0,2475.0,2475.0,6037.0,2.0,33969206.0,...,3101.0,96110.0,0.0,1924.0,141633.0,293249.0,2016.0,151616.0,4060.23,6.037535e+13
1253214,1253214,17072208,2.0,4.0,2.0,1508.0,1508.0,6111.0,2.0,34293885.0,...,2061.0,97081.0,7.0,1929.0,129868.0,303027.0,2016.0,173159.0,3275.16,6.111002e+13
509533,509533,11048086,2.0,3.0,2.0,1298.0,1298.0,6037.0,2.0,34255080.0,...,3101.0,96371.0,0.0,1953.0,99347.0,185219.0,2016.0,85872.0,2274.42,6.037110e+13
2781116,2781116,12467777,1.0,2.0,1.0,1465.0,1465.0,6037.0,1.0,33821829.0,...,3101.0,96241.0,0.0,1939.0,91248.0,456244.0,2016.0,364996.0,5466.24,6.037572e+13
1720117,1720117,11698041,1.0,2.0,1.0,1384.0,1384.0,6037.0,1.0,33989541.0,...,3101.0,96024.0,0.0,1922.0,18247.0,42385.0,2016.0,24138.0,624.30,6.037235e+13
222288,222288,10714423,2.0,3.0,2.0,1887.0,1887.0,6037.0,2.0,34220246.0,...,3101.0,96339.0,0.0,1959.0,214000.0,591000.0,2016.0,377000.0,7247.52,6.037113e+13
378196,378196,11883260,1.0,3.0,1.0,2071.0,2071.0,6037.0,1.0,34105747.0,...,3101.0,96280.0,0.0,1914.0,134771.0,673872.0,2016.0,539101.0,8231.35,6.037481e+13
915534,915534,12027799,2.0,2.0,2.0,1325.0,1325.0,6037.0,2.0,34104300.0,...,3101.0,96027.0,0.0,1965.0,136913.0,541661.0,2016.0,404748.0,6598.20,6.037194e+13


Sweet, our data looks so much better. __We freed up $\approx$ .7+ GB of memory__! Let's press forward.

In [36]:
original_mem_usage = df.memory_usage(index=False).sum()/1_000_000_000
subset_mem_usage = df_subset.memory_usage(index=False).sum()/1_000_000_000

free_memory = original_mem_usage - subset_mem_usage
print(f"{free_memory:.2f}GB of free memory!")

0.79GB of free memory!


### Subset of Zillow data
- Dropped columns missing more than 9.14% of values

In [37]:
# Will return. Retrieving and reproducing data dictionary for these columns.
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 26 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   id                            int64  
 1   parcelid                      int64  
 2   bathroomcnt                   float64
 3   bedroomcnt                    float64
 4   calculatedbathnbr             float64
 5   calculatedfinishedsquarefeet  float64
 6   finishedsquarefeet12          float64
 7   fips                          float64
 8   fullbathcnt                   float64
 9   latitude                      float64
 10  longitude                     float64
 11  lotsizesquarefeet             float64
 12  propertycountylandusecode     object 
 13  propertylandusetypeid         float64
 14  rawcensustractandblock        float64
 15  regionidcity                  float64
 16  regionidcounty                float64
 17  regionidzip                   float64
 18  roomcnt               

In [38]:
# Create a function to clean our Zillow dataset
# Note: Create a wrangle function to acquire and prepare the dataset
def prepare_zillow(df):
    '''
    Signature: prepare_zillow(df) -> pandas.core.frame.DataFrame
    Docstring:
    Prepare the zillow dataset for data EDA

    Return DataFrame of zillow dataset

    Parameters
    ----------
    df : pandas.core.frame.DataFrame
    df is the Zillow dataset stored as `zillow.csv`

    Returns
    -------
    DataFrame of the zillow dataset


    Examples
    --------
    To copy the code hold ALT + SHIFT and drag your cursor from the first line to the last.

    >>> df = pd.read_csv('./data/raw/zillow.csv')
    >>>
    >>> nulls = df.isnull().sum()
    >>> non_nulls = df.notnull().sum()
    >>> total_values = nulls + non_nulls
    >>>
    >>> pct_missing = (nulls/total_values).sort_values(ascending=False)
    >>> pct_missing_chart = pct_missing.apply("{0:.2%}".format)
    >>>
    >>> print('Percentage of values missing per column')
    >>> print('-' * 39)
    >>> print(f"{pct_missing_chart}")
    '''
    
    # 1. Drop columns with missing values
    # Calculate the number of missing values and non-null values.
    nulls = df.isnull().sum()
    non_nulls = df.notnull().sum()

    # Get the total number values in each column.
    total_values = nulls + non_nulls

    # Create a variable to store the percentage of missing values in each column.
    # .3358 is a hard coded value from the original analysis.
    # To reproduce use the code in the docstring
    pct_missing = nulls/total_values
    columns_to_drop = pct_missing[pct_missing > .3358].index.to_list()
    
    # drop columns missing more than 33.58% of data.
    df = df.drop(columns=columns_to_drop)
    
    # drop duplicate columns and index column
    duplicate_columns_to_drop = ['calculatedbathnbr', 'finishedsquarefeet12', 'id']
    df = df.drop(columns=duplicate_columns_to_drop)
    
    df = df[df.propertylandusetypeid.isin([261, 262, 273, 279])]
    
    return df

In [39]:
test = prepare_zillow(df)

In [40]:
# Sweet, the function works!
test.sample(10)

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
424840,12323435,2.0,4.0,1098.0,6037.0,2.0,33895313.0,-118178206.0,5537.0,0100,...,3101.0,96220.0,0.0,1955.0,97367.0,194734.0,2016.0,97367.0,2530.51,6.037554e+13
735688,11730092,2.0,4.0,1566.0,6037.0,2.0,34031278.0,-118307235.0,6349.0,0100,...,3101.0,95999.0,0.0,1978.0,134381.0,223388.0,2016.0,89007.0,2786.21,6.037222e+13
785648,13081719,2.0,3.0,1278.0,6037.0,2.0,34006176.0,-117874195.0,5999.0,0100,...,3101.0,96525.0,0.0,1963.0,85739.0,166198.0,2016.0,80459.0,2318.16,6.037408e+13
2197898,11963155,1.0,2.0,823.0,6037.0,1.0,34108119.0,-118221288.0,4918.0,0100,...,3101.0,96046.0,0.0,1948.0,137934.0,456245.0,2016.0,318311.0,5601.96,6.037185e+13
1319617,12381841,2.0,4.0,1450.0,6037.0,2.0,33983543.0,-118118218.0,7829.0,0100,...,3101.0,96192.0,0.0,1942.0,119900.0,400000.0,2016.0,280100.0,12237.35,6.037532e+13
711867,13949432,2.0,3.0,1101.0,6059.0,2.0,33748523.0,-117968544.0,7286.0,122,...,1286.0,96990.0,6.0,1958.0,49693.0,266154.0,2016.0,216461.0,3482.92,6.059089e+13
2426878,14031490,2.5,4.0,2286.0,6059.0,2.0,33734434.0,-117957127.0,6000.0,122,...,1286.0,96990.0,8.0,1966.0,146139.0,418061.0,2016.0,271922.0,5047.24,6.059099e+13
1068374,12946743,2.0,3.0,1216.0,6037.0,2.0,34042015.0,-117952199.0,6080.0,0100,...,3101.0,96488.0,0.0,1955.0,88553.0,329155.0,2016.0,240602.0,4013.90,6.037407e+13
674909,11455980,2.0,3.0,2236.0,6037.0,2.0,33968643.0,-118384926.0,10875.0,0100,...,3101.0,96026.0,0.0,1950.0,74033.0,141113.0,2016.0,67080.0,1819.10,6.037276e+13
2799032,13890748,1.0,3.0,1008.0,6059.0,1.0,33614147.0,-117908034.0,4000.0,122,...,1286.0,96981.0,5.0,1942.0,28213.0,115848.0,2016.0,87635.0,2251.28,6.059063e+13


In [41]:
df = prepare_zillow(df)

In [42]:
df.shape

(2152864, 23)

In [43]:
df.fips.value_counts()

6037.0    1431812
6059.0     555077
6111.0     165975
Name: fips, dtype: int64

In [17]:
# Data dictionary template
numeric_columns = df.select_dtypes(exclude='O')

for column in numeric_columns.columns.to_list():
    print(column, numeric_columns[column].min(), numeric_columns[column].max())

parcelid 10711725 169601949
bathroomcnt 0.0 32.0
bedroomcnt 0.0 25.0
calculatedfinishedsquarefeet 1.0 952576.0
fips 6037.0 6111.0
fullbathcnt 1.0 32.0
latitude 33339912.0 34819650.0
longitude -119475780.0 -117554316.0
lotsizesquarefeet 100.0 371000512.0
propertylandusetypeid 261.0 279.0
rawcensustractandblock 60371011.101 61110091.003010996
regionidcity 3491.0 396556.0
regionidcounty 1286.0 3101.0
regionidzip 95982.0 399675.0
roomcnt 0.0 96.0
yearbuilt 1801.0 2016.0
structuretaxvaluedollarcnt 1.0 66404932.0
taxvaluedollarcnt 1.0 98428909.0
assessmentyear 2000.0 2016.0
landtaxvaluedollarcnt 1.0 88921951.0
taxamount 1.85 1337755.86
censustractandblock -1.0 483030105084015.0


square feet of home, number of bedrooms, and number of bathrooms.

In [44]:
def prepare_zillow_mvp(df):
    '''
    
    '''
    df = df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt']]
    return df

In [45]:
df = pd.read_csv('zillow.csv')
df = prepare_zillow_mvp(df)

In [50]:
df.shape

(2985217, 4)

In [46]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
0,0.0,0.0,NaN,9.0
1,0.0,0.0,NaN,27516.0
2,0.0,0.0,73026.0,1434941.0
3,0.0,0.0,5068.0,1174475.0
4,0.0,0.0,1776.0,440101.0


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 4 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bathroomcnt                   float64
 1   bedroomcnt                    float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
dtypes: float64(4)
memory usage: 91.1 MB


In [49]:
df.isna().sum()

bathroomcnt                      2957
bedroomcnt                       2945
calculatedfinishedsquarefeet    45097
taxvaluedollarcnt               34266
dtype: int64

In [52]:
df = df.dropna()

In [53]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
2,0.0,0.0,73026.0,1434941.0
3,0.0,0.0,5068.0,1174475.0
4,0.0,0.0,1776.0,440101.0
5,0.0,0.0,2400.0,287634.0
7,0.0,0.0,3611.0,698984.0


In [55]:
df = df[(df['bathroomcnt'] > 0) & (df['bathroomcnt'] > 0)]

In [56]:
df.describe()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
count,2.868299e+06,2.868299e+06,2.868299e+06,2.868299e+06
mean,2.303907e+00,3.214631e+00,1.803914e+03,4.409267e+05
std,1.003256e+00,1.135802e+00,1.109706e+03,6.092388e+05
min,5.000000e-01,0.000000e+00,1.000000e+00,1.000000e+00
25%,2.000000e+00,3.000000e+00,1.216000e+03,1.929850e+05
50%,2.000000e+00,3.000000e+00,1.576000e+03,3.242720e+05
75%,3.000000e+00,4.000000e+00,2.133000e+03,5.155435e+05
max,3.200000e+01,2.500000e+01,9.525760e+05,9.018846e+07
